In [107]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

def calculate_angle(point1, point2, point3):
    """
    Calculate the angle between the line from point1 to point2 and from point2 to point3.
    """
    vector1 = np.array(point1) - np.array(point2)
    vector2 = np.array(point3) - np.array(point2)

    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    angle = math.acos(dot_product / (magnitude1 * magnitude2))
    return np.degrees(angle)

def filter_blue_lines(image):
    """
    Filter out blue lines in the image using color thresholding.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the blue color range in HS
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([255, 255, 255])

    # Create a mask for blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image to keep only the blue regions
    blue_only = cv2.bitwise_and(image, image, mask=mask)

    return blue_only

def find_contours_and_angle(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Detect edges using Canny
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 200, 255)

    dilate = cv2.dilate(canny, (5,5), iterations=5)
    erode  = cv2.erode(dilate, (5,5), iterations=7)

    # Find contours
    contours, _ = cv2.findContours(erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Assume the largest contour is the shape of interest
        contour = max(contours, key=cv2.contourArea)

        # Approximate the contour to get the vertices of the shape
        epsilon = 0.1 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) >= 2:
            cv2.drawContours(img, [approx], -1, (0, 255, 0), 2)
            for point in approx:
                cv2.circle(img, tuple(point[0]), 5, (0, 0, 255), -1)

            # Let's say we want to calculate the angle at the first vertex
            point1 = approx[0][0]
            point2 = approx[1][0]
            point3 = approx[2][0]

            # Calculate the angle
            angle = calculate_angle(point1, point2, point3)
            print(f"The angle is: {angle:.2f} degrees")

            # Display the image with the contour
            cv2.imshow('Blue Lines Contour', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        else:
            print("Not enough vertices to calculate an angle.")
    else:
        print("No contours found.")

# Example usage
find_contours_and_angle('/Users/hanna m/machinelearning/deep_learning/cv/analyse_geometery/Q1-1.png')

The angle is: 120.39 degrees
